In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
!nvidia-smi

Sat Apr 13 16:32:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   27C    P0              51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
%%bash
nvidia-smi --query-gpu=index,memory.used,memory.total,memory.free,utilization.gpu --format=csv,noheader | awk -F, '
BEGIN {
    printf "GPU,Memory [%%],Free,Utilization [%%]\n"
}
{
    printf "%s,%6.2f,%-12s,%-12s\n", $1, ($2/$3)*100, $4, $5
}' | sed 's|\s%||g' | awk -F, '
BEGIN {
    OFS="\t";   # Output field separator, adjust as needed
}
{
    for (i=1; i<=NF; i++) {
        printf "%-20s", $i;   # Adjust the width (20 in this example) as needed
    }
    print "";   # Print a new line after each row
}'

GPU                 Memory [%]          Free                Utilization [%]     
0                     0.00               40339 MiB           0                  
1                     0.00               40339 MiB           0                  
2                     0.00               40339 MiB           0                  
3                     0.00               40339 MiB           0                  
4                     0.00               40339 MiB           0                  
5                     0.00               40339 MiB           0                  
6                     0.00               40339 MiB           0                  
7                     0.00               40339 MiB           0                  


In [4]:
import pandas as pd

In [5]:
import os, sys
sys.path.append('../')
os.chdir('../')



In [6]:
%cd Tugas Akhir/indonlu

/workspace/Tugas Akhir/indonlu


In [7]:
!pwd

/workspace/Tugas Akhir/indonlu


In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [9]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 139 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 215 kB 29.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.3 MB 76.6 MB/s eta 0:00:01


# 1. Prepare for finetuning NER-Grit 

In [9]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
from tqdm import tqdm

from transformers import BertConfig, BertTokenizer, XLMRobertaTokenizerFast
from nltk.tokenize import word_tokenize

from modules.word_classification import BertForWordClassification
from utils.forward_fn import forward_word_classification
from utils.metrics import ner_metrics_fn
from utils.data_utils import NerGritDataset, NerDataLoader

In [10]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [11]:

# Set random seed
set_seed(26092020)

In [12]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-large-p2',ignore_mismatched_sizes=True)
config = BertConfig.from_pretrained('indobenchmark/indobert-large-p2', ignore_mismatched_sizes=True)
config.num_labels = NerGritDataset.NUM_LABELS

# Instantiate model
model = BertForWordClassification.from_pretrained('indobenchmark/indobert-large-p2', config=config,ignore_mismatched_sizes=True)

Some weights of BertForWordClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model

BertForWordClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12

In [14]:
w2i, i2w = NerGritDataset.LABEL2INDEX, NerGritDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'I-PERSON': 0, 'B-ORGANISATION': 1, 'I-ORGANISATION': 2, 'B-PLACE': 3, 'I-PLACE': 4, 'O': 5, 'B-PERSON': 6}
{0: 'I-PERSON', 1: 'B-ORGANISATION', 2: 'I-ORGANISATION', 3: 'B-PLACE', 4: 'I-PLACE', 5: 'O', 6: 'B-PERSON'}


In [15]:
def word_subword_tokenize(sentence, tokenizer):
    # Add CLS token
    subwords = [tokenizer.cls_token_id]
    subword_to_word_indices = [-1] # For CLS

    # Add subwords
    for word_idx, word in enumerate(sentence):
        subword_list = tokenizer.encode(word, add_special_tokens=False)
        subword_to_word_indices += [word_idx for i in range(len(subword_list))]
        subwords += subword_list

    # Add last SEP token
    subwords += [tokenizer.sep_token_id]
    subword_to_word_indices += [-1]

    return subwords, subword_to_word_indices

In [16]:
train_dataset_path = './dataset/nergrit_ner-grit/train_preprocess.txt'
valid_dataset_path = './dataset/nergrit_ner-grit/valid_preprocess.txt'
test_dataset_path = './dataset/nergrit_ner-grit/test_preprocess_masked_label.txt'

In [17]:
train_dataset = NerGritDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = NerGritDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = NerGritDataset(test_dataset_path, tokenizer, lowercase=True)

train_loader = NerDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
valid_loader = NerDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  
test_loader = NerDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [18]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)
model = model.cuda()

In [19]:
# Train
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = ner_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_word_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = ner_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = ner_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

(Epoch 1) TRAIN LOSS:1.1988 LR:0.00002000: 100%|██████████| 105/105 [00:16<00:00,  6.38it/s]


(Epoch 1) TRAIN LOSS:1.1988 ACC:0.82 F1:0.05 REC:0.04 PRE:0.11 LR:0.00002000


VALID LOSS:1.2187 ACC:0.85 F1:0.06 REC:0.04 PRE:0.12: 100%|██████████| 14/14 [00:05<00:00,  2.63it/s]


(Epoch 1) VALID LOSS:1.2187 ACC:0.85 F1:0.06 REC:0.04 PRE:0.12


(Epoch 2) TRAIN LOSS:0.9218 LR:0.00002000: 100%|██████████| 105/105 [00:15<00:00,  6.93it/s]


...

(Epoch 4) VALID LOSS:0.5869 ACC:0.95 F1:0.72 REC:0.73 PRE:0.71


(Epoch 5) TRAIN LOSS:0.5492 LR:0.00002000: 100%|██████████| 105/105 [00:15<00:00,  6.73it/s]


(Epoch 5) TRAIN LOSS:0.5492 ACC:0.96 F1:0.76 REC:0.78 PRE:0.74 LR:0.00002000


VALID LOSS:0.5587 ACC:0.96 F1:0.74 REC:0.79 PRE:0.70: 100%|██████████| 14/14 [00:05<00:00,  2.68it/s]

(Epoch 5) VALID LOSS:0.5587 ACC:0.96 F1:0.74 REC:0.79 PRE:0.70


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 2. Prepare for gazetteer

In [21]:
%cd /workspace/Tugas Akhir/text_processing/location_extraction_src/lib/gazetteer
kec_to_id = {}
id_to_kec = {}
city_to_id = {}
id_to_city = {}
id_to_prov = {}


with open('prov_gazetteer.csv') as f:
    lines = f.readlines()
    j = -1
    for line in lines:
        j+=1
        if j ==0:
            continue
        id_, prov_ = line.split(',')[0], line.split(',')[1][:-1]
        id_to_prov[id_] = prov_
f.close()

with open('kota_kab_gazetteer.csv') as f:
    lines = f.readlines()
    j = -1
    for line in lines:
        j+=1
        if j ==0:
            continue
        id_, city_ = line.split(',')[0], line.split(',')[1][:-1]
        city_to_id[city_] = id_
        id_to_city[id_] = city_
f.close()

with open('kecamatan_gazetteer.csv') as f:
    lines = f.readlines()
    j = -1
    #print(lines)
    for line in lines:
        j+=1
        if j ==0:
            continue
        id_, kec_ = line.split(',')[0], line.split(',')[1][:-1]
        #print(line)
        kec_to_id[kec_.upper()] = id_
        id_to_kec[id_] = kec_.upper()
f.close()



/workspace/Tugas Akhir/text_processing/location_extraction_src/lib/gazetteer


# 3. Location Extraction pipeline with NER

In [22]:
import nltk.data

def split_sentences(text):
    # Menggunakan PunktSentenceTokenizer untuk Bahasa Indonesia
    #print(text)
    temp =text[0]
    for i in range(1,len(text)-1):
        if text[i]=="." and text[i-1]!=' ' and text[i+1]!=' ':
            temp+='. '
        else:
            temp+=text[i]
    temp+=text[-1]
    ret = []
    for j in temp.split('. '):
        if (j.lower() == j.upper()):
            continue
        if len(j)==0:
            continue
        if j[-1]!='.':
            ret.append(j+'.')
        else:
            ret.append(j)
    return ret

def get_hasil_ner_tiap_sentence(teks):
    text = word_tokenize(teks)
    subwords, subword_to_word_indices = word_subword_tokenize(text, tokenizer)

    subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
    subword_to_word_indices = torch.LongTensor(subword_to_word_indices).view(1, -1).to(model.device)
    logits = model(subwords, subword_to_word_indices)[0]

    preds = torch.topk(logits, k=1, dim=-1)[1].squeeze().cpu().numpy()
    labels = [i2w[preds[i]] for i in range(len(preds))]
    final_text = []
    final_labels = []
    cek_kanan = True
    for i in range(len(text)-1):
        if not cek_kanan:
            cek_kanan = True
            continue
        kata_kunci = text[i]
        kanan = text[i+1]
        cek = kata_kunci+' '+kanan
        if cek.lower() in keyword_multi:
            final_text.append(cek)
            cek_kanan = False
        else:
            final_text.append(kata_kunci)
        final_labels.append(labels[i])
    final_text.append(text[-1])
    final_labels.append(labels[-1])
    #print(final_text)
    #print(final_labels)
    return pd.DataFrame({'words': final_text, 'label': final_labels})

def get_entitas_lokasi(teks):
    #print("WOI",teks)
    list_of_sentence = split_sentences(teks)
    #print(list_of_sentence)
    df_get_entitas = get_hasil_ner_tiap_sentence(list_of_sentence[0])
    

    for i in range(1, len(list_of_sentence)):
        df_get_entitas = pd.concat([df_get_entitas, get_hasil_ner_tiap_sentence(list_of_sentence[i])], ignore_index=True)

    lst = []
    i = 0
    temp = ""
    #print(df.head(10))
    while i<len(df_get_entitas):
        token = df_get_entitas['words'].loc[i]
        pos_tag = df_get_entitas['label'].loc[i]
        i+=1
        if len(pos_tag)<2 or pos_tag[-5:] != 'PLACE':
            if temp != "":
                lst.append(temp)
            temp = ""
            continue
        if temp != "":
            temp += " "+ token
        else:
            temp = token
    return lst



In [57]:
def get_entitas_lokasi_baru(teks,id_bencana,eksperimen, threshold, is_filtering_on = True):
    #print("WOI",teks)
    if len(df_ngecek[df_ngecek['id']==id_bencana])==0:
        return []
    list_of_sentence = split_sentences(teks)
    #print(list_of_sentence)
    
    df_get_entitas = get_hasil_ner_tiap_sentence(list_of_sentence[0])
    lst_nomor_sentence = [f'({0}, {i})' for i in range(len(df_get_entitas))]

    for i in range(1, len(list_of_sentence)):
        tempo_df = get_hasil_ner_tiap_sentence(list_of_sentence[i])
        df_get_entitas = pd.concat([df_get_entitas, tempo_df], ignore_index=True)
        lst_nomor_sentence.extend([f'({i}, {j})' for j in range(len(tempo_df))])
    lst = []
    i = 0
    temp = ""
    filter_cek  = True
    #print(df.head(10))
    df_get_entitas['id_sentence_token'] = lst_nomor_sentence
    while i<len(df_get_entitas):
        
        indeks_st = df_get_entitas['id_sentence_token'].loc[i]
        #print(indeks_st)
        token = df_get_entitas['words'].loc[i]
        pos_tag = df_get_entitas['label'].loc[i]
        i+=1
        
        if len(pos_tag)<2 or pos_tag[-5:] != 'PLACE':
            if temp != "":
                if filter_cek:
                    lst.append(temp)
                    if is_filtering_on:
                        filter_cek = False
            temp = ""
            continue
            
        
        if temp != "":
            temp += " "+ token
            if eksperimen ==0:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['selisih_depth'])[0]<=threshold:
                    filter_cek = True 
            elif eksperimen ==1:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['jarak_keyword_lokasi_undir'])[0]<=threshold:
                    filter_cek = True
            elif eksperimen ==2:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['jarak_keyword_lokasi_dir'])[0]<=threshold:
                    filter_cek = True 
            else:
                filter_cek = filter_cek or list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['is_connect_to_key'])[0]
        else:
            temp = token
            if eksperimen ==0:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['selisih_depth'])[0]<=threshold:
                    filter_cek = True
            elif eksperimen ==1:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['jarak_keyword_lokasi_undir'])[0]<=threshold:
                    filter_cek = True 
            elif eksperimen ==2:
                if list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['jarak_keyword_lokasi_dir'])[0]<=threshold:
                    filter_cek = True 
            else:
                filter_cek = filter_cek or list(df_ngecek[df_ngecek['id']==id_bencana][df_ngecek['indeks_sentence_token']==indeks_st]['is_connect_to_key'])[0]
    return lst

# 4. Build dependency parsing

In [24]:
keyword_bencana = {
    'angin_topan':['badai', 'puting beliung', 'angin topan', 'tornado', 'angin kencang'],
    'banjir':['banjir'],
    'erupsi':['vulkanik', 'erupsi', 'letusan', 'awan panas', 'lava'],
    'gempa':['gempa', 'tektonik'],
    'karhutla':['kebakaran hutan', 'kebakaran lahan', 'titik panas'],
    'kekeringan':['kekeringan'],
    'longsor':['longsor'],
    'tsunami':['tsunami']
}
tambahan_katakunci = {
	"guncang":["guncang"],
	"landa":["landa"],
	"timpa":["menimpa", "timpa"],
	"terpa":["terpa", "menerpa"],
	"terjang":["terjang", "menerjang"],
	"hantam":["hantam"],
	"kataganti":["bencana", "ini", "itu", "tersebut"],	
    "dampak":["terdampak", "dampak"],
    "rusak":["rusak", "merusak","merusakkan", "hancur"],
    "tumbang":["tumbang","menumbang"],
    "roboh":["rubuh", "roboh"],
    "genang":["genang", "tergenang"],
    "kena":["terkena", "kena"]
}

In [25]:
tmp_keyword = []
for kunci in tambahan_katakunci.keys():
    tmp_keyword.extend(tambahan_katakunci[kunci])
keyword_bencana['tambahan'] = tmp_keyword
keyword_bencana

{'angin_topan': ['badai',
  'puting beliung',
  'angin topan',
  'tornado',
  'angin kencang'],
 'banjir': ['banjir'],
 'erupsi': ['vulkanik', 'erupsi', 'letusan', 'awan panas', 'lava'],
 'gempa': ['gempa', 'tektonik'],
 'karhutla': ['kebakaran hutan', 'kebakaran lahan', 'titik panas'],
 'kekeringan': ['kekeringan'],
 'longsor': ['longsor'],
 'tsunami': ['tsunami'],
 'tambahan': ['guncang',
  'landa',
  'menimpa',
  'timpa',
  'terpa',
  'menerpa',
  'terjang',
  'menerjang',
  'hantam',
  'bencana',
  'ini',
  'itu',
  'tersebut',
  'terdampak',
  'dampak',
  'rusak',
  'merusak',
  'merusakkan',
  'hancur',
  'tumbang',
  'menumbang',
  'rubuh',
  'roboh',
  'genang',
  'tergenang',
  'terkena',
  'kena']}

In [26]:
keyword_multi = ['puting beliung', 'angin topan', 'angin kencang', 'awan panas','kebakaran hutan', 'kebakaran lahan', 'titik panas']

In [27]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
%load_ext autoreload
%autoreload 2

# DP

In [30]:
!pwd

/workspace/Tugas Akhir/text_processing/location_extraction_src/lib/gazetteer


In [31]:
%cd /workspace/Tugas Akhir

/workspace/Tugas Akhir


In [32]:
import sys
sys.path.append("/workspace/indolem/dependency_parsing")

In [33]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
###########

tokenizer_pos = AutoTokenizer.from_pretrained("ageng-anugrah/indobert-large-p2-finetuned-pos")
modul_pos = AutoModelForTokenClassification.from_pretrained("ageng-anugrah/indobert-large-p2-finetuned-pos")


In [34]:
!pip install conllu
!pip install overrides

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 4.1 Prediction function for producing dependency tree

## get the sentence and list of postag of each word

In [35]:
def predict(model, tokenizer, sentence):# TO GET POSTAG AND 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(sentence,
                    is_split_into_words = True,
                    return_offsets_mapping=True, 
                    return_tensors="pt",
                    padding='max_length', 
                    truncation=True, 
                    max_length=512)
    
    model.to(device)
    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [model.config.id2label[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
        #only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])
        else:
            continue
    
    return sentence, prediction

## get dependency tree

In [36]:
from dep_parser import get_dependency
def predict_dependency(words, tags):# untuk menghasilkan tree, dengan masukan list of word dan list of tags
    a, b = words, tags 
    zipped_word_label = []
    instance2indexpos =  {
            "_PAD_POS": 0,
            "_ROOT_POS": 1,
            "_END_POS": 2,
            "PROPN": 3,
            "AUX": 4,
            "DET": 5,
            "NOUN": 6,
            "PRON": 7,
            "VERB": 8,
            "ADP": 9,
            "PUNCT": 10,
            "ADV": 11,
            "CCONJ": 12,
            "SCONJ": 13,
            "NUM": 14,
            "ADJ": 15,
            "PART": 16,
            "SYM": 17,
            "X": 18
        }
    mapping = {
        "PRP": "PRON",
        "NN": "NOUN",
        "NNP": "PROPN",
        "CD": "NUM",
        "VB": "VERB",
        "RB": "ADV",
        "DT": "DET"
    }     
    for i, j in zip(a,b):
        j = mapping.get(j[2:], j[2:])
        if j not in instance2indexpos.keys():
            j = "X"
        zipped_word_label.append((i, j))
        
    return get_dependency(zipped_word_label)

In [37]:
def is_keyword(word, bencana):
    list_all_keyword = [k for k in keyword_bencana[bencana]]
    list_all_keyword.extend(keyword_bencana['tambahan'])
    benar = False
    for katakunci in list_all_keyword:
        if katakunci.lower() in word.lower():
            benar = True
            if katakunci == "ini" or katakunci == "itu":
                benar = katakunci==word.lower()
            break
    return benar

## 4.2 Defining class of graph for rulebased experiment

In [38]:

from collections import defaultdict
import math
#This class represents a directed graph using adjacency list representation
class Graph:

	def __init__(self,vertices):
		self.V= vertices #No. of vertices
		self.graph = defaultdict(list) # default dictionary to store graph
		self.undir = defaultdict(list)
	# function to add an edge to graph
	def addEdge(self,u,v):
		self.graph[u].append(v)
		self.undir[u].append(v)
		self.undir[v].append(u)
	
	# Use BFS to check path between s and d
	def get_bfs(self, s, is_directed):
		# Mark all the vertices as not visited
		visited =[-1]*(self.V)

		# Create a queue for BFS
		queue=[]

		# Mark the source node as visited and enqueue it
		queue.append(s)
		visited[s] = 0
		adj_list = self.graph if is_directed else self.undir

		while queue:

			#Dequeue a vertex from queue
			n = queue.pop(0)
			
			# If this adjacent node is the destination node,
			# then return true

			# Else, continue to do BFS
			for i in adj_list[n]:
				if visited[i] == -1:
					queue.append(i)
					visited[i] = visited[n]+1
		# If BFS is complete without visited d
		return visited
	def get_depth(self, d):
		visited = self.get_bfs(-1, False)
		return visited[d]
	def get_difference_depth(self, s, d):
		depth = self.get_depth(s)-self.get_depth(d)
		return depth if depth>=0 else -depth
	def get_distance(self, s, d, is_directed = False):
		visited1 = self.get_bfs(s, is_directed)
		visited2 = self.get_bfs(d, is_directed)
		if visited1[d]>0:
			return visited1[d]
		if visited2[s]>0:
			return visited2[s]
		return 1e9

def construct_graph(dp_tree):
	g = Graph(len(dp_tree)+1)
	for i in range(len(dp_tree)):
		node_of_dp = dp_tree[i]
		g.addEdge(node_of_dp[0], node_of_dp[-1])
	return g

### get list of dependency tree from each document

In [39]:
def get_list_dependency_parsing_from_doc(teks, bencana):
    list_index_of_loc = []
    list_index_of_keyword = []
    list_of_sentence = split_sentences(teks)
    list_of_sentence_tokenized = []
    list_of_sentence_tokenized_label = []
    list_of_dependency_parsing_each_sentence = []
    num_sent = 0
    for sentence in list_of_sentence:
        #print("sent bef", num_sent)
        is_contain_loc = False
        is_contain_keyword = False
        sentence_df = get_hasil_ner_tiap_sentence(sentence)
        list_of_token = list(sentence_df['words'])
        list_of_label = list(sentence_df['label'])
        list_of_sentence_tokenized.append(list_of_token)
        list_of_sentence_tokenized_label.append(list_of_label)
        num_of_token = 0
        #print("sent aft", num_sent)
        for token in list_of_token:
            #print("tok bef", num_of_token)
            if is_keyword(token, bencana):
                list_index_of_keyword.append((num_sent, num_of_token))
                is_contain_keyword = True
            label = list_of_label[num_of_token]
            if len(label)>=2 and label[-5:]== 'PLACE':
                is_contain_loc = True
                list_index_of_loc.append((num_sent, num_of_token))
                
            #print("tok aft", num_of_token)
            num_of_token+=1
        #print("mungkinkah lama di sini?")
        if  is_contain_keyword :
            list_of_token, list_of_tag = predict(modul_pos, tokenizer_pos, list_of_token)

            list_of_dependency_parsing_each_sentence.append(predict_dependency(list_of_token, list_of_tag))
        #print("mungkin")
        else:
            list_of_dependency_parsing_each_sentence.append([])
        num_sent+=1
    return list_index_of_loc, list_index_of_keyword, list_of_sentence_tokenized, list_of_dependency_parsing_each_sentence


 

### get the nearest distance of each location on text to set disaster keyword 
suppose there are n keyword and m detected location, this function will return the distance of each m detected location to nearest keyword. If there is no keyword that can reached by some location or the condition doesnt meet, it will return infinite

In [40]:
def get_experiment_distance_doc(teks, bencana):
    list_index_of_loc, list_index_of_keyword, list_of_sentence_tokenized, list_of_dependency_parsing_each_sentence = get_list_dependency_parsing_from_doc(teks, bencana)
    dct_graph = {}
    for i in range(len(list_of_dependency_parsing_each_sentence)):
        dp = list_of_dependency_parsing_each_sentence[i]
        if dp ==[]:
            continue
        else:
            dct_graph[i] = construct_graph(dp)
    dct_distance_lokasi_each_dokumen = {}
    for in_keyword in list_index_of_keyword:
        for in_lokasi  in list_index_of_loc:
            if in_keyword[0]!=in_lokasi[0]:
                continue
            else:
                temp_graph = dct_graph[in_keyword[0]]
                try:
                    selisih_depth, jarak_keyword_lokasi_undir, jarak_keyword_lokasi_dir, is_connect_to_key = tuple(dct_distance_lokasi_each_dokumen[in_lokasi])
                    
                    selisih_depth = min(temp_graph.get_difference_depth(in_keyword[-1],in_lokasi[-1]), selisih_depth)
                    jarak_keyword_lokasi_dir = min(temp_graph.get_distance(in_keyword[-1],in_lokasi[-1], True), jarak_keyword_lokasi_dir)
                    jarak_keyword_lokasi_undir = min(temp_graph.get_distance(in_keyword[-1],in_lokasi[-1]), jarak_keyword_lokasi_undir)
                    is_connect_to_key = True if jarak_keyword_lokasi_dir<1e3 else False
                    dct_distance_lokasi_each_dokumen[in_lokasi] = [selisih_depth, jarak_keyword_lokasi_undir, jarak_keyword_lokasi_dir, is_connect_to_key]
                    
                except:
                    selisih_depth = temp_graph.get_difference_depth(in_keyword[-1],in_lokasi[-1])
                    jarak_keyword_lokasi_dir = temp_graph.get_distance(in_keyword[-1],in_lokasi[-1], True)
                    jarak_keyword_lokasi_undir = temp_graph.get_distance(in_keyword[-1],in_lokasi[-1])
                    is_connect_to_key = True if jarak_keyword_lokasi_dir<1e3 else False
                    dct_distance_lokasi_each_dokumen[in_lokasi] = [selisih_depth, jarak_keyword_lokasi_undir, jarak_keyword_lokasi_dir, is_connect_to_key]
    for in_lokasi  in list_index_of_loc:
        try:
            checkk = dct_distance_lokasi_each_dokumen[in_lokasi]
        except:
            dct_distance_lokasi_each_dokumen[in_lokasi] = [1e9, 1e9, 1e9, False]
    return dct_distance_lokasi_each_dokumen

### function for preproces list of distance in some article become dataframe 

In [41]:
def get_df_dct_experiment_distance_each_doc(id_berita, berita, bencana, part):
    
    dct = get_experiment_distance_doc(berita, bencana)
    list_id_berita = [id_berita for i in range(len(dct))]
    list_part = [part for i in range(len(dct))]
    daftar_kunci = list(dct.keys())
    daftar_kunci.sort()
    
    list_selisih_depth = []
    list_jarak_keyword_lokasi_undir = []
    list_jarak_keyword_lokasi_dir = []
    list_is_connect_to_key = []
    for kunci in daftar_kunci:
        lst_dis = dct[kunci]
        selisih_depth = lst_dis[0]
        jarak_keyword_lokasi_undir = lst_dis[1]
        jarak_keyword_lokasi_dir = lst_dis[2]
        is_connect_to_key = lst_dis[3]
        
        list_selisih_depth.append(selisih_depth)
        list_jarak_keyword_lokasi_undir.append(jarak_keyword_lokasi_undir)
        list_jarak_keyword_lokasi_dir.append(jarak_keyword_lokasi_dir)
        list_is_connect_to_key.append(is_connect_to_key)
    
    data_temp = {
        'id':list_id_berita,
        'indeks_sentence_token':daftar_kunci,
        'selisih_depth':list_selisih_depth,
        'jarak_keyword_lokasi_undir':list_jarak_keyword_lokasi_undir,
        'jarak_keyword_lokasi_dir':list_jarak_keyword_lokasi_dir,
        'is_connect_to_key':list_is_connect_to_key,
        'part':list_part,
    }
    return pd.DataFrame(data_temp)

### load data for experiment

In [42]:
%cd dataset

/workspace/Tugas Akhir/dataset


In [43]:
df_all = pd.read_csv(f'./angin_topan/angin_topan_train.csv')
lst_part = ['train' for i in range(len(df_all))]
for bencana in ['angin_topan', 'banjir', 'erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']:
    for partition in ['train', 'dev', 'test']:
        if bencana =='angin_topan' and partition == 'train':
            continue
        temp_df = pd.read_csv(f'./{bencana}/{bencana}_{partition}.csv')
        df_all = pd.concat([df_all, temp_df], ignore_index=True)
        for i in range(len(temp_df)):
            lst_part.append(partition)
df_all['part'] = lst_part
df_all = df_all[['id','tc_ori','label', 'part']]
df_all = df_all[df_all['label']=="Y"]

In [44]:
bencana_df = []
for i in range(len(df_all)):
    bencana_df.append(df_all['id'].iloc[i][:-5])
df_all['bencana'] = bencana_df

In [45]:
df_all

id  \
0     angin_topan_0002   
1     angin_topan_0003   
3     angin_topan_0006   
4     angin_topan_0007   
5     angin_topan_0008   
...                ...   
9373      tsunami_1107   
9376      tsunami_1142   
9379      tsunami_1146   
9386      tsunami_0536   
9391      tsunami_1109   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [58]:
!pwd

/workspace/Tugas Akhir/precomputed_location_extraction


In [47]:
%cd ..
%cd precomputed_location_extraction

/workspace/Tugas Akhir
/workspace/Tugas Akhir/precomputed_location_extraction


In [ ]:
bencana_distance_all = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
bencana_distance = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
prev_bencana = 'angin_topan'
df_train = df_all[df_all['part']=='train']
with open('indeks yang gagal dihitung.txt', 'a') as xx:
    
    for i in range(len(df_train)):
        print(f'{i} from {len(df_train)}')
        id_bencana  = df_train['id'].iloc[i]
        berita  = df_train['tc_ori'].iloc[i]
        bencana  = df_train['bencana'].iloc[i]
        part  = df_train['part'].iloc[i]
        try:
            tempe_df = get_df_dct_experiment_distance_each_doc(id_bencana, berita, bencana, part)
            if prev_bencana!= bencana:
                print('-------------GANTI BENCANA ---------')
                
                bencana_distance.to_csv(f'./precomputed_location_extraction/loc_with_distance_pre_compute_{prev_bencana}.csv', index = False) 
                bencana_distance = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
                prev_bencana = bencana
            bencana_distance_all= pd.concat([bencana_distance_all, tempe_df], ignore_index=True)
            bencana_distance= pd.concat([bencana_distance, tempe_df], ignore_index=True)
        except:
            xx.writes(str(id_bencana))
            

    bencana_distance_all.to_csv(f'./precomputed_location_extraction/loc_with_distance_pre_compute_all.csv', index = False) 
xx.close

In [60]:
bencana_distance_all = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
bencana_distance = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
prev_bencana = 'angin_topan'
df_test = df_all[df_all['part']=='test']
with open('indeks yang gagal dihitung.txt', 'a') as xx:
    
    for i in range(len(df_test)):
        print(f'{i} from {len(df_test)}')
        id_bencana  = df_test['id'].iloc[i]
        berita  = df_test['tc_ori'].iloc[i]
        bencana  = df_test['bencana'].iloc[i]
        part  = df_test['part'].iloc[i]
        try:
            tempe_df = get_df_dct_experiment_distance_each_doc(id_bencana, berita, bencana, part)
            if prev_bencana!= bencana:
                print('-------------GANTI BENCANA ---------')
                
                bencana_distance.to_csv(f'./precomputed_location_extraction/loc_with_distance_pre_compute_{prev_bencana}_test.csv', index = False) 
                bencana_distance = pd.DataFrame([], columns=['id', 'indeks_sentence_token', 'selisih_depth', 'jarak_keyword_lokasi_undir', 'jarak_keyword_lokasi_dir', 'is_connect_to_key','part'])
                prev_bencana = bencana
            bencana_distance_all= pd.concat([bencana_distance_all, tempe_df], ignore_index=True)
            bencana_distance= pd.concat([bencana_distance, tempe_df], ignore_index=True)
        except:
            xx.writes(str(id_bencana))
            

    bencana_distance_all.to_csv(f'./precomputed_location_extraction/loc_with_distance_pre_compute_all_test.csv', index = False) 
xx.close

0 from 375
1 from 375
2 from 375
3 from 375
4 from 375
5 from 375
6 from 375
7 from 375
8 from 375
9 from 375
10 from 375
11 from 375
12 from 375
13 from 375
14 from 375
15 from 375
16 from 375
17 from 375
18 from 375
19 from 375
20 from 375
21 from 375
22 from 375
23 from 375
24 from 375
25 from 375
26 from 375
27 from 375
28 from 375
29 from 375
30 from 375
31 from 375
32 from 375
33 from 375
34 from 375
35 from 375
36 from 375
37 from 375
38 from 375
39 from 375
40 from 375
41 from 375
42 from 375
43 from 375
44 from 375
45 from 375
46 from 375
-------------GANTI BENCANA ---------
47 from 375
48 from 375
49 from 375
50 from 375
51 from 375
52 from 375
53 from 375
54 from 375
55 from 375
56 from 375
57 from 375
58 from 375
59 from 375
60 from 375
61 from 375
62 from 375
63 from 375
64 from 375
65 from 375
66 from 375
67 from 375
68 from 375
69 from 375
70 from 375
71 from 375
72 from 375
73 from 375
74 from 375
75 from 375
76 from 375
77 from 375
78 from 375
79 from 375
80 from 375
8

<function TextIOWrapper.close()>

### getting wider information of location with gazetteer, 
e.g *pare* (one of kecamatan in kediri) to (pare, kediri, jawa timur) 

In [48]:
def get_from_city(word):
    id_city = "Unknown"
    try:
        id_city = city_to_id[word.upper()]
    except:
        pass
    if "Unknown"== id_city:
        return id_city
    else:
        prov = id_to_prov[id_city[:2]]
        return  word.upper(), prov
    
def get_from_kec(word):
    id_kecamatan = "Unknown"
    try:
        
        id_kecamatan = kec_to_id[word.upper()]
    except:
        pass
    if "Unknown"== id_kecamatan:
        return id_kecamatan
    else:
        city = id_to_city[id_kecamatan[:5]]
        prov = id_to_prov[id_kecamatan[:2]]
        return word.upper(), city, prov
def search_loc_gazetteer(word):
    #kecamatan
    result = "Unknown"
    lst_loc =  word.split()
    if len(lst_loc)>=4:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+4])
            if get_from_kec(loc)!= "Unknown":
                return  get_from_kec(loc)
    if len(lst_loc)>=4:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+4])
            if get_from_city(loc)!= "Unknown":
                temp = get_from_city(loc)
                return "Unknown", temp[0], temp[1]
    if len(lst_loc)>=3:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+3])
            if get_from_kec(loc)!= "Unknown":
                return  get_from_kec(loc)
    if len(lst_loc)>=3:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+3])
            if get_from_city(loc)!= "Unknown":
                temp = get_from_city(loc)
                return "Unknown", temp[0], temp[1]
    if len(lst_loc)>=2:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+2])
            if get_from_kec(loc)!= "Unknown":
                return get_from_kec(loc)
    if len(lst_loc)>=2:
        for j in range(len(lst_loc)):
            loc = ' '.join(lst_loc[j:j+2])
            if get_from_city(loc)!= "Unknown":
                temp = get_from_city(loc)
                return "Unknown", temp[0], temp[1]
    if len(lst_loc)>=1:
        for loc in lst_loc:
            if loc == 'Kecamatan' or loc == 'Kota' or loc == 'Kabupaten' or loc == 'Provinsi':
                continue
            if get_from_kec(loc)!= "Unknown":
                return  get_from_kec(loc)
    if len(lst_loc)>=1:
        for loc in lst_loc:
            if loc == 'Kecamatan' or loc == 'Kota' or loc == 'Kabupaten' or loc == 'Provinsi':
                continue
            if get_from_city(loc)!= "Unknown":
                temp = get_from_city(loc)
                return "Unknown", temp[0], temp[1]
    return "Unknown", "Unknown", "Unknown"

def extract_loc_with_ner_and_gazetteer(berita,id_bencana,eksperimen, threshold, is_filtering_on = True):
    isi_kota = get_entitas_lokasi_baru(berita,id_bencana,eksperimen, threshold,is_filtering_on)
    #print(isi_kota)
    new_lokasi = list(set(isi_kota))
    #print(new_lokasi)
    freq_lokasi = []
    for lokasi in new_lokasi:
        jumlah = len(lokasi.split())
        freq_lokasi.append((jumlah,lokasi))
    freq_lokasi.sort()
    
    indeks = [True for i in range(len(freq_lokasi))]
    for i in range(len(freq_lokasi)):
        tup = freq_lokasi[i]
        lokasi = tup[1]
        for j in range(i+1, len(freq_lokasi)):
            if lokasi in freq_lokasi[j][1]:
                indeks[i] = False
                
    news_lokasi = []
    for i in range(len(indeks)):
        if indeks[i]:
            news_lokasi.append(freq_lokasi[i][1])     
        
    res_lok = set()
    for lok in news_lokasi:
        res_lok.add(search_loc_gazetteer(lok))
    res_gazetteer = list(res_lok)
    #print("isi",res_gazetteer)
    indeks_res = [True for i in range(len(res_gazetteer))]
    for i in range(len(res_gazetteer)):
        lok1 = res_gazetteer[i]
        if lok1 == ('Unknown', 'Unknown', 'Unknown'):
            indeks_res[i] = False
            continue
        elif lok1[0] != 'Unknown':
            continue
        elif lok1[0] == 'Unknown' and lok1[1] == 'Unknown':
            for j in range(len(res_gazetteer)):
                if i==j:
                    continue
                lok2 = res_gazetteer[j]
                if lok2[1] != 'Unknown' and lok1[2] == lok2[2]:
                    indeks_res[i] = False
                    break
        elif  lok1[0] == 'Unknown':
            for j in range(len(res_gazetteer)):
                if i==j:
                    continue
                lok2 = res_gazetteer[j]
                if lok2[0] != 'Unknown' and lok1[2] == lok2[2] and lok1[1] == lok2[1]:
                    indeks_res[i] = False
                    break
    f = [res_gazetteer[i] for i in range(len(res_gazetteer)) if indeks_res[i]]
    final_res = [(loc[0].upper(), loc[1], loc[2]) for loc in f]
    return final_res

In [49]:
df_ngecek = pd.read_csv(f'./loc_with_distance_pre_compute_angin_topan.csv')
for bencana in [ 'angin_topan', 'banjir', 'erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']:
    tempe_df = pd.read_csv(f'./loc_with_distance_pre_compute_{bencana}.csv')
    df_ngecek= pd.concat([df_ngecek, tempe_df], ignore_index=True)


In [50]:
for bencana in [ 'angin_topan', 'banjir', 'erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']:
    tempe_df = pd.read_csv(f'./loc_with_distance_pre_compute_{bencana}_test.csv')
    df_ngecek= pd.concat([df_ngecek, tempe_df], ignore_index=True)

In [51]:
df_ngecek

,id,indeks_sentence_token,selisih_depth,jarak_keyword_lokasi_undir,jarak_keyword_lokasi_dir,is_connect_to_key,part
0,angin_topan_0002,"(0, 2)",1.000000e+09,1.000000e+09,1.000000e+09,False,train
1,angin_topan_0002,"(1, 7)",1.000000e+09,1.000000e+09,1.000000e+09,False,train
2,angin_topan_0002,"(4, 19)",1.000000e+09,1.000000e+09,1.000000e+09,False,train
3,angin_topan_0002,"(4, 20)",1.000000e+09,1.000000e+09,1.000000e+09,False,train
4,angin_topan_0002,"(4, 21)",1.000000e+09,1.000000e+09,1.000000e+09,False,train
...,...,...,...,...,...,...,...
73081,tsunami_1109,"(13, 13)",2.000000e+00,4.000000e+00,1.000000e+09,False,test
73082,tsunami_1109,"(13, 14)",3.000000e+00,5.000000e+00,1.000000e+09,False,test
73083,tsunami_1109,"(52, 13)",0.000000e+00,4.000000e+00,1.000000e+09,False,test
73084,tsunami_1109,"(52, 15)",0.000000e+00,4.000000e+00,1.000000e+09,False,test


In [52]:
temp_lst = []
for i in range(len(df_ngecek)):
    temp_lst.append(df_ngecek['id'].iloc[i][:-5])
df_ngecek['bencana'] = temp_lst

In [63]:
!pip install xlrd
!pip install openpyxl 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 249 kB 3.3 MB/s eta 0:00:01


### load ground truth of location

In [53]:
lst_file = ["angin_topan_loc_time.xlsx","banjir_loc_time.xlsx", "erupsi_loc_time.xlsx", "gempa_loc_time.xlsx", "karhutla_loc_time.xlsx","karhutla_loc_time.xlsx", "kekeringan_loc_time.xlsx", "longsor_loc_time.xlsx", "tsunami_loc_time.xlsx"]

In [54]:
import pandas as pd
lsc_col = ['id', 'Positif Bencana Alam', 'Tanggal Kejadian', 'Lokasi ADM1',
       'Lokasi ADM2', 'Lokasi ADM3', 'Lokasi Lainnya']
df_gt = ""
for bencana in lst_file:
    read_file = "../dataset/location/"
    read_file+=bencana
    
    if len(df_gt)==0:
        df_gt= pd.read_excel(read_file)[lsc_col]  
    else:
        tmp_dgt =  pd.read_excel(read_file)
        df_gt = pd.concat([df_gt,tmp_dgt[lsc_col]], ignore_index=True)
df_gt = df_gt.fillna({'Lokasi ADM1':'UNKNOWN','Lokasi ADM2':'UNKNOWN','Lokasi ADM3':'UNKNOWN'})
lst_agg = []
for i in range(len(df_gt)):
    a1 = df_gt['Lokasi ADM1'].iloc[i].upper()
    if a1[-1] == ' ':
        a1 = a1[:-1]
    if a1[0] == ' ':
        a1 = a1[1:]
    a2 = df_gt['Lokasi ADM2'].iloc[i].upper()
    if a2[-1] == ' ':
        a2 = a2[:-1]
    if a2[0] == ' ':
        a2 = a2[1:]
    a3 = df_gt['Lokasi ADM3'].iloc[i].upper()
    if a3[-1] == ' ':
        a3 = a3[:-1]
    if a3[0] == ' ':
        a3 = a3[1:]
    lst_agg.append((a3,a2,a1))
df_gt['loc_aggregate'] = lst_agg

df_gt = df_gt[df_gt['Positif Bencana Alam']!='X']


In [55]:
df_gt=df_gt.reset_index(drop=True)

In [56]:
def get_loc_gt_with_id(id_bencana):
    return list(set(df_gt[df_gt['id'] == id_bencana]['loc_aggregate'])-{('UNKNOWN', 'UNKNOWN', 'UNKNOWN')})


### function for predict some id and with given condition of experiment

In [61]:
def get_pred_loc_with_id(id_bencana, eksperimen,threshold, is_filtering_on = True):
    doc = list(df_all[df_all['id'] == id_bencana]['tc_ori'])[0]
    while True:
        try:
            if eksperimen>=3:
                threshold = 0
            return extract_loc_with_ner_and_gazetteer(doc,id_bencana,eksperimen, threshold,is_filtering_on)
        except:
            doc = '. '.join(doc.split('. ')[:-1])


### evaluate performance precision, recall, and F1

In [63]:
def get_eval_loc_with_id(id_bencana, eksperimen,threshold, is_filtering_on = True):#ret prec, rec, f1
    gt = get_loc_gt_with_id(id_bencana)
    pred = get_pred_loc_with_id(id_bencana, eksperimen,threshold,is_filtering_on)
    ls_id = [id_bencana for i in range(len(pred))]
    
    TP = len(set(gt).intersection(set(pred)))
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    
    if TP == 0:
        return TP, FP, FN, 0,0,0
    prec = TP/(TP+FP)
    rec = TP/(TP+FN)
    f1 = 0
    if prec*rec!=0:
        f1 = 2/(1/prec+1/rec)
    return TP, FP, FN, prec, rec, f1, pd.DataFrame({'id':ls_id, 'loc_pred':list(pred)})


In [64]:
def get_df_hasil_eval_each_doc(id_bencana, eksperimen,threshold):
    TP, FP, FN, prec, rec, f1, df_tempe_tempe = get_eval_loc_with_id(id_bencana, eksperimen,threshold)
    return pd.DataFrame([[id_bencana,TP, FP, FN, prec, rec, f1]], columns=['id', 'TP', 'FP', 'FN', 'Prec', 'Rec','F1']), df_tempe_tempe

In [67]:
%cd ..

/workspace/Tugas Akhir


In [72]:
df_all_pelatihan = df_all[df_all['part']!='test']
df_all_pelatihan.reset_index(drop=True)

,id,tc_ori,label,part,bencana
0,angin_topan_0002,"Pencarian Nelayan Simeulue Dihentikan. Setelah sepekan operasi, pencarian dua nelayan Simeulue akhirnya dihentikan. Petugas dan relawan berhenti mencari sejak pukul 00.00 WIB, Jumat (11/11). Nelayan yang belum berhasil ditemukan Andi Manalu (25) dan Erianto (31), kedua berasal dari Desa Suka Maju, Kecamatan Simeulue Timur. Semalam secara resmi dihentikan operasi gabungan pencarian terhadap dua nelayan kita, kata Ardiansyah, Dantim SAR operasi gabungan pencarian. Ia turut didampingi Ikhsan Mikaris, Kepala BPBD Simeulue. Walau telah dihentikan, bila mendapatkan informasi keberadaan dua nelayan hilang itu operasi kembali digelar. Meskipun secara resmi telah dihentikan operasi pencarian itu, namun pihak kita tidak melarang kepada keluarga dan masyarakat yang masih berupaya ke laut untuk melakukan pencarian, kata Ikhsan Mikaris. Ia menyebutkan sebelumnya area operasi pencarian diperluas hingga 60 mil laut, hingga mencapai pulau Bangkaru, Aceh Singkil serta penyisiran berulang kali di perairan laut sejumlah pulau-pulau kecil, perairan laut pulau Babi dan pulau Lasia, Kecamatan Teupah Selatan. Selama pelaksanaan operasi gabungan pencarian nelayan itu, telah melibatkan puluhan personil, dari SAR, BPBD, TNI AL, relawan dan masyarakat, dengan menggunakan armada perahu nelayan, kapal boat ikan serta Kapal Angkatan Laut Sinabang milik TNI Lanal Simeulue. Sebelumnya Perahu Bagan Ikan KM Tiga Putri itu diterjang badai dan gelombang, sehingga menyebabkan terjadi kebocoran serius, kemudian tenggelam di perairan laut pulau Simanaha, Kecamatan Simeulue Timur, sekitar pukul 02.00 WIB, Jumat (4/11). Di atas perahu bagan Ikan KM Tiga Putri itu, sembilan nelayan berusaha menyelamatkan diri. Namun yang selamat hanya, Bagus (25), Wen (30), Ibok (27) dan Endi (27). Mereka lantas memberitahukan pada warga lainnya, lima rekannya telah hilang usai perahu disapu badai. Setelah dilakukan operasi gabungan pencarian, tiga orang ditemukan Syaid Suhaidi (36), Rijal Setiawan (36) dan Budiono (26), ditemukan terdampar di pulau Babi, setelah 25 jam hanyut, sekitar 40 mil dari lokasi kejadian. Hingga kemarin, dua korban belum ditemukan sampai operasi pencarian dihentikan. (ahi/mai)",Y,train,angin_topan
1,angin_topan_0003,"Pohon Tumbang Timpa Warga Korban Meninggal Dalam Kapal BRR. Seorang pengendara motor dilaporkan mengalami luka berat hingga tewas akibat tertimpa pohon tumbang di ruas jalan Jurong Perikanan, Gampong Kuta Timu, Kecamatan Sukakarya Sabang, Rabu (3/1). Selain menewaskan seorang pengendara motor, pohon juga menimpa satu unit rumah dan merobohkan dua tiang listrik milik PLN. Kapolres Sabang AKBP Slamet Wahyudi, mengatakan korban Hasra Rudi (38), warga Jurong Putro Ijo, Gampong Aneuk Laot, Kecamatan Sukakarya, Kota Sabang. Ia mengalami luka berat tertimpa dahan pohon tumbang, meninggal dunia di kapal BRR saat hendak dirujuk ke rumah Sakit Zainoel Abidin Banda Aceh. Peristiwa tersebut terjadi saat kondisi dalam keadaan hujan disertai angin kencang. Disaat itulah korban Hasra Rudi, pedagang dengan menggunakan sepeda motor jenis SUPRA X125 berwarna Biru, No. Pol BL 4014 AM melaju dari arah Aneuk Laot menuju ke kota. Namun naas, ketika sepeda motor korban melintas di ruas Jalan Jurong Perikanan tiba-tiba saja tumbang pohon berukuran besar. Kita perkirakan korban saat itu panik tidak lagi mampu mengelakkan kendaraannya dari timpaan dahan kayu yang tumbang, jelasnya. Akibatnya kepala korban bagian belakang tertimpa dahan kayu, dan sepeda motor korban juga ikut tertimpa. Masyarakat yang mengetahui terjadinya peristiwa itu langsung menolong korban. Membawanya ke RSU Kota Sabang untuk mendapatkan pertolongan medis, kata Kapolres. Katanya, saat korban dibawa ke rumah sakit kondisinya sudah dalam keadaan tidak sadar. Sehingga diambil tindakan pihak Rumah Sakit Umum Sabang penanganan pasien keadaan darurat sementara, lalu dilanjutkan dengan memberikan rujukan atas korban ke RSU Zainoel Abidin di Banda Aceh. Untuk 

In [ ]:
for eksperimen in [0, 3, 1, 2]:
    for threshold in range(0, 10):
        if eksperimen==3 and threshold>0:
            continue
        if (eksperimen == 2 or eksperimen ==1) and threshold ==0:
            continue
        print('Eksperimen', eksperimen, 'threshold', threshold)
        hasil_eksperimen = pd.DataFrame([], columns=['id', 'TP', 'FP', 'FN', 'Prec', 'Rec','F1'])
        for i in range(len(df_all_pelatihan)):
            id_berita = df_all_pelatihan['id'].iloc[i]
            print(id_berita, f'{i} of {len(df_all_pelatihan)}')
            tempe_df = get_df_hasil_eval_each_doc(id_berita, eksperimen,threshold)
            hasil_eksperimen= pd.concat([hasil_eksperimen, tempe_df], ignore_index=True)
        print('selesai sebuah threshold')
        if eksperimen ==3:
            hasil_eksperimen.to_csv(f'./prediction/result_information_extraction/Hasil_Evaluasi_NER_dan_DP_ISCONNECTED_KeyLoc.csv')
        else:
            nama_eks = ''
            if eksperimen == 0:
                nama_eks = 'selisih_depth'
            elif eksperimen == 1:
                nama_eks = 'jarak_keyword_lokasi_undir'
            elif eksperimen == 2:
                nama_eks = 'jarak_keyword_lokasi_dir'
            hasil_eksperimen.to_csv(f'./prediction/result_information_extraction/Hasil_Evaluasi_NER_dan_DP_{nama_eks}_KeyLoc_threshold={threshold}.csv')
    print('selesai sebuah eksperimen')
            
            
        
        
        

Eksperimen 0 threshold 0
angin_topan_0002 0 of 1837
angin_topan_0003 1 of 1837
angin_topan_0006 2 of 1837
angin_topan_0007 3 of 1837
angin_topan_0008 4 of 1837
angin_topan_0019 5 of 1837
angin_topan_0020 6 of 1837
angin_topan_0021 7 of 1837
angin_topan_0027 8 of 1837
angin_topan_0028 9 of 1837
angin_topan_0032 10 of 1837
angin_topan_0046 11 of 1837
angin_topan_0047 12 of 1837
angin_topan_0048 13 of 1837
angin_topan_0049 14 of 1837
angin_topan_0055 15 of 1837
angin_topan_0065 16 of 1837
angin_topan_0070 17 of 1837
angin_topan_0091 18 of 1837
angin_topan_0096 19 of 1837
angin_topan_0108 20 of 1837
angin_topan_0119 21 of 1837
angin_topan_0121 22 of 1837
angin_topan_0127 23 of 1837
angin_topan_0140 24 of 1837
angin_topan_0156 25 of 1837
angin_topan_0163 26 of 1837
angin_topan_0168 27 of 1837
angin_topan_0170 28 of 1837
angin_topan_0175 29 of 1837
angin_topan_0179 30 of 1837
angin_topan_0183 31 of 1837
angin_topan_0189 32 of 1837
angin_topan_0193 33 of 1837
angin_topan_0208 34 of 1837
angin

banjir_0602 306 of 1837
banjir_0608 307 of 1837
banjir_0617 308 of 1837
banjir_0619 309 of 1837
banjir_0620 310 of 1837
banjir_0621 311 of 1837
banjir_0625 312 of 1837
banjir_0630 313 of 1837
banjir_0632 314 of 1837
banjir_0634 315 of 1837
banjir_0648 316 of 1837
banjir_0671 317 of 1837
banjir_0678 318 of 1837
banjir_0687 319 of 1837
banjir_0703 320 of 1837
banjir_0705 321 of 1837
banjir_0709 322 of 1837
banjir_0713 323 of 1837
banjir_0738 324 of 1837
banjir_0755 325 of 1837
banjir_0769 326 of 1837
banjir_0776 327 of 1837
banjir_0788 328 of 1837
banjir_0804 329 of 1837
banjir_0817 330 of 1837
banjir_0820 331 of 1837
banjir_0824 332 of 1837
banjir_0831 333 of 1837
banjir_0833 334 of 1837
banjir_0837 335 of 1837
banjir_0850 336 of 1837
banjir_0852 337 of 1837
banjir_0857 338 of 1837
banjir_0861 339 of 1837
banjir_0874 340 of 1837
banjir_0879 341 of 1837
banjir_0888 342 of 1837
banjir_0901 343 of 1837
banjir_0906 344 of 1837
banjir_0912 345 of 1837
banjir_0916 346 of 1837
banjir_0917 347 

gempa_0564 651 of 1837
gempa_0567 652 of 1837
gempa_0568 653 of 1837
gempa_0581 654 of 1837
gempa_0585 655 of 1837
gempa_0613 656 of 1837
gempa_0616 657 of 1837
gempa_0618 658 of 1837
gempa_0621 659 of 1837
gempa_0639 660 of 1837
gempa_0643 661 of 1837
gempa_0646 662 of 1837
gempa_0647 663 of 1837
gempa_0653 664 of 1837
gempa_0660 665 of 1837
gempa_0664 666 of 1837
gempa_0674 667 of 1837
gempa_0676 668 of 1837
gempa_0679 669 of 1837
gempa_0682 670 of 1837
gempa_0690 671 of 1837
gempa_0695 672 of 1837
gempa_0704 673 of 1837
gempa_0708 674 of 1837
gempa_0714 675 of 1837
gempa_0731 676 of 1837
gempa_0735 677 of 1837
gempa_0737 678 of 1837
gempa_0759 679 of 1837
gempa_0760 680 of 1837
gempa_0764 681 of 1837
gempa_0767 682 of 1837
gempa_0769 683 of 1837
gempa_0770 684 of 1837
gempa_0778 685 of 1837
gempa_0781 686 of 1837
gempa_0783 687 of 1837
gempa_0784 688 of 1837
gempa_0791 689 of 1837
gempa_0792 690 of 1837
gempa_0795 691 of 1837
gempa_0801 692 of 1837
gempa_0804 693 of 1837
gempa_0814 

karhutla_0796 981 of 1837
karhutla_0801 982 of 1837
karhutla_0803 983 of 1837
karhutla_0804 984 of 1837
karhutla_0809 985 of 1837
karhutla_0811 986 of 1837
karhutla_0813 987 of 1837
karhutla_0815 988 of 1837
karhutla_0816 989 of 1837
karhutla_0825 990 of 1837
karhutla_0826 991 of 1837
karhutla_0830 992 of 1837
karhutla_0833 993 of 1837
karhutla_0836 994 of 1837
karhutla_0840 995 of 1837
karhutla_0846 996 of 1837
karhutla_0848 997 of 1837
karhutla_0849 998 of 1837
karhutla_0858 999 of 1837
karhutla_0859 1000 of 1837
karhutla_0865 1001 of 1837
karhutla_0869 1002 of 1837
karhutla_0879 1003 of 1837
karhutla_0884 1004 of 1837
karhutla_0885 1005 of 1837
karhutla_0887 1006 of 1837
karhutla_0890 1007 of 1837
karhutla_0891 1008 of 1837
karhutla_0892 1009 of 1837
karhutla_0894 1010 of 1837
karhutla_0903 1011 of 1837
karhutla_0918 1012 of 1837
karhutla_0921 1013 of 1837
karhutla_0924 1014 of 1837
karhutla_0925 1015 of 1837
karhutla_0939 1016 of 1837
karhutla_0941 1017 of 1837
karhutla_0943 1018 o

kekeringan_1052 1274 of 1837
kekeringan_1056 1275 of 1837
kekeringan_1067 1276 of 1837
kekeringan_1071 1277 of 1837
kekeringan_1082 1278 of 1837
kekeringan_1090 1279 of 1837
kekeringan_1091 1280 of 1837
kekeringan_1097 1281 of 1837
kekeringan_1098 1282 of 1837
kekeringan_0028 1283 of 1837
kekeringan_0036 1284 of 1837
kekeringan_0067 1285 of 1837
kekeringan_0132 1286 of 1837
kekeringan_0152 1287 of 1837
kekeringan_0188 1288 of 1837
kekeringan_0219 1289 of 1837
kekeringan_0280 1290 of 1837
kekeringan_0301 1291 of 1837
kekeringan_0344 1292 of 1837
kekeringan_0383 1293 of 1837
kekeringan_0388 1294 of 1837
kekeringan_0430 1295 of 1837
kekeringan_0457 1296 of 1837
kekeringan_0476 1297 of 1837
kekeringan_0490 1298 of 1837
kekeringan_0536 1299 of 1837
kekeringan_0659 1300 of 1837
kekeringan_0675 1301 of 1837
kekeringan_0686 1302 of 1837
kekeringan_0708 1303 of 1837
kekeringan_0762 1304 of 1837
kekeringan_0794 1305 of 1837
kekeringan_0837 1306 of 1837
kekeringan_0894 1307 of 1837
kekeringan_108

...

karhutla_0809 985 of 1837
karhutla_0811 986 of 1837
karhutla_0813 987 of 1837
karhutla_0815 988 of 1837
karhutla_0816 989 of 1837
karhutla_0825 990 of 1837
karhutla_0826 991 of 1837
karhutla_0830 992 of 1837
karhutla_0833 993 of 1837
karhutla_0836 994 of 1837
karhutla_0840 995 of 1837
karhutla_0846 996 of 1837
karhutla_0848 997 of 1837
karhutla_0849 998 of 1837
karhutla_0858 999 of 1837
karhutla_0859 1000 of 1837
karhutla_0865 1001 of 1837
karhutla_0869 1002 of 1837
karhutla_0879 1003 of 1837
karhutla_0884 1004 of 1837
karhutla_0885 1005 of 1837
karhutla_0887 1006 of 1837
karhutla_0890 1007 of 1837
karhutla_0891 1008 of 1837
karhutla_0892 1009 of 1837
karhutla_0894 1010 of 1837
karhutla_0903 1011 of 1837
karhutla_0918 1012 of 1837
karhutla_0921 1013 of 1837
karhutla_0924 1014 of 1837
karhutla_0925 1015 of 1837
karhutla_0939 1016 of 1837
karhutla_0941 1017 of 1837
karhutla_0943 1018 of 1837
karhutla_0946 1019 of 1837
karhutla_0947 1020 of 1837
karhutla_0949 1021 of 1837
karhutla_0951 10

kekeringan_1082 1278 of 1837
kekeringan_1090 1279 of 1837
kekeringan_1091 1280 of 1837
kekeringan_1097 1281 of 1837
kekeringan_1098 1282 of 1837
kekeringan_0028 1283 of 1837
kekeringan_0036 1284 of 1837
kekeringan_0067 1285 of 1837
kekeringan_0132 1286 of 1837
kekeringan_0152 1287 of 1837
kekeringan_0188 1288 of 1837
kekeringan_0219 1289 of 1837
kekeringan_0280 1290 of 1837
kekeringan_0301 1291 of 1837
kekeringan_0344 1292 of 1837
kekeringan_0383 1293 of 1837
kekeringan_0388 1294 of 1837
kekeringan_0430 1295 of 1837
kekeringan_0457 1296 of 1837
kekeringan_0476 1297 of 1837
kekeringan_0490 1298 of 1837
kekeringan_0536 1299 of 1837
kekeringan_0659 1300 of 1837
kekeringan_0675 1301 of 1837
kekeringan_0686 1302 of 1837
kekeringan_0708 1303 of 1837
kekeringan_0762 1304 of 1837
kekeringan_0794 1305 of 1837
kekeringan_0837 1306 of 1837
kekeringan_0894 1307 of 1837
kekeringan_1083 1308 of 1837
kekeringan_0973 1309 of 1837
kekeringan_0454 1310 of 1837
longsor_0005 1311 of 1837
longsor_0007 1312

tsunami_0124 1590 of 1837
tsunami_0128 1591 of 1837
tsunami_0154 1592 of 1837
tsunami_0158 1593 of 1837
tsunami_0175 1594 of 1837
tsunami_0182 1595 of 1837
tsunami_0183 1596 of 1837
tsunami_0190 1597 of 1837
tsunami_0195 1598 of 1837
tsunami_0197 1599 of 1837
tsunami_0207 1600 of 1837
tsunami_0211 1601 of 1837
tsunami_0212 1602 of 1837
tsunami_0214 1603 of 1837
tsunami_0216 1604 of 1837
tsunami_0218 1605 of 1837
tsunami_0222 1606 of 1837
tsunami_0223 1607 of 1837
tsunami_0224 1608 of 1837
tsunami_0234 1609 of 1837
tsunami_0249 1610 of 1837
tsunami_0252 1611 of 1837
tsunami_0260 1612 of 1837
tsunami_0263 1613 of 1837
tsunami_0265 1614 of 1837
tsunami_0268 1615 of 1837
tsunami_0272 1616 of 1837
tsunami_0274 1617 of 1837
tsunami_0276 1618 of 1837
tsunami_0277 1619 of 1837
tsunami_0280 1620 of 1837
tsunami_0285 1621 of 1837
tsunami_0287 1622 of 1837
tsunami_0288 1623 of 1837
tsunami_0291 1624 of 1837
tsunami_0292 1625 of 1837
tsunami_0294 1626 of 1837
tsunami_0296 1627 of 1837
tsunami_0297

angin_topan_0458 62 of 1837
angin_topan_0459 63 of 1837
angin_topan_0463 64 of 1837
angin_topan_0464 65 of 1837
angin_topan_0466 66 of 1837
angin_topan_0474 67 of 1837
angin_topan_0480 68 of 1837
angin_topan_0515 69 of 1837
angin_topan_0524 70 of 1837
angin_topan_0528 71 of 1837
angin_topan_0549 72 of 1837
angin_topan_0550 73 of 1837
angin_topan_0572 74 of 1837
angin_topan_0573 75 of 1837
angin_topan_0580 76 of 1837
angin_topan_0584 77 of 1837
angin_topan_0588 78 of 1837
angin_topan_0613 79 of 1837
angin_topan_0623 80 of 1837
angin_topan_0632 81 of 1837
angin_topan_0641 82 of 1837
angin_topan_0660 83 of 1837
angin_topan_0662 84 of 1837
angin_topan_0671 85 of 1837
angin_topan_0683 86 of 1837
angin_topan_0692 87 of 1837
angin_topan_0695 88 of 1837
angin_topan_0697 89 of 1837
angin_topan_0701 90 of 1837
angin_topan_0703 91 of 1837
angin_topan_0713 92 of 1837
angin_topan_0715 93 of 1837
angin_topan_0737 94 of 1837
angin_topan_0744 95 of 1837
angin_topan_0750 96 of 1837
angin_topan_0761 97 

banjir_1081 379 of 1837
banjir_1082 380 of 1837
banjir_1088 381 of 1837
banjir_1091 382 of 1837
banjir_1092 383 of 1837
banjir_1096 384 of 1837
banjir_1097 385 of 1837
banjir_1098 386 of 1837
banjir_1100 387 of 1837
banjir_1103 388 of 1837
banjir_1105 389 of 1837
banjir_1112 390 of 1837
banjir_1114 391 of 1837
banjir_1115 392 of 1837
banjir_1132 393 of 1837
banjir_1135 394 of 1837
banjir_1139 395 of 1837
banjir_1145 396 of 1837
banjir_1146 397 of 1837
banjir_1150 398 of 1837
banjir_1151 399 of 1837
banjir_1161 400 of 1837
banjir_1163 401 of 1837
banjir_1164 402 of 1837
banjir_1177 403 of 1837
banjir_1183 404 of 1837
banjir_1186 405 of 1837
banjir_1209 406 of 1837
banjir_1212 407 of 1837
banjir_1220 408 of 1837
banjir_1226 409 of 1837
banjir_1240 410 of 1837
banjir_1241 411 of 1837
banjir_0009 412 of 1837
banjir_0035 413 of 1837
banjir_0074 414 of 1837
banjir_0097 415 of 1837
banjir_0116 416 of 1837
banjir_0134 417 of 1837
banjir_0155 418 of 1837
banjir_0189 419 of 1837
banjir_0217 420 

In [111]:
df_all_test = df_all[df_all['part']=='test']
df_all_test.reset_index(drop=True)

id  \
0    angin_topan_0005   
1    angin_topan_0040   
2    angin_topan_0051   
3    angin_topan_0066   
4    angin_topan_0107   
..                ...   
370      tsunami_1107   
371      tsunami_1142   
372      tsunami_1146   
373      tsunami_0536   
374      tsunami_1109   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

### get df eval test according result train

In [191]:
for eksperimen in [0, 3, 1, 2]:
    threshold = 5
    if eksperimen==1 or eksperimen == 2 :
        threshold = 8
    
    print('Eksperimen', eksperimen, 'threshold', threshold)
    hasil_eksperimen = pd.DataFrame([], columns=['id', 'TP', 'FP', 'FN', 'Prec', 'Rec','F1'])
    for i in range(len(df_all_test)):
        id_berita = df_all_test['id'].iloc[i]
        print(id_berita, f'{i} of {len(df_all_test)}')
        tempe_df = get_df_hasil_eval_each_doc(id_berita, eksperimen,threshold)
        hasil_eksperimen= pd.concat([hasil_eksperimen, tempe_df], ignore_index=True)
    print('selesai sebuah threshold')
    if eksperimen ==3:
        hasil_eksperimen.to_csv(f'./prediction/result_information_extraction/Hasil_Evaluasi_TEST_NER_dan_DP_ISCONNECTED_KeyLoc.csv',index = False)
    else:
        nama_eks = ''
        if eksperimen == 0:
            nama_eks = 'selisih_depth'
        elif eksperimen == 1:
            nama_eks = 'jarak_keyword_lokasi_undir'
        elif eksperimen == 2:
            nama_eks = 'jarak_keyword_lokasi_dir'
        hasil_eksperimen.to_csv(f'./prediction/result_information_extraction/Hasil_Evaluasi_TEST_NER_dan_DP_{nama_eks}_KeyLoc_threshold={threshold}.csv',index = False)
    print('selesai sebuah eksperimen')

Eksperimen 0 threshold 5
angin_topan_0005 0 of 375
angin_topan_0040 1 of 375
angin_topan_0051 2 of 375
angin_topan_0066 3 of 375
angin_topan_0107 4 of 375
angin_topan_0143 5 of 375
angin_topan_0144 6 of 375
angin_topan_0181 7 of 375
angin_topan_0203 8 of 375
angin_topan_0206 9 of 375
angin_topan_0224 10 of 375
angin_topan_0244 11 of 375
angin_topan_0297 12 of 375
angin_topan_0301 13 of 375
angin_topan_0356 14 of 375
angin_topan_0445 15 of 375
angin_topan_0470 16 of 375
angin_topan_0491 17 of 375
angin_topan_0543 18 of 375
angin_topan_0562 19 of 375
angin_topan_0583 20 of 375
angin_topan_0625 21 of 375
angin_topan_0677 22 of 375
angin_topan_0696 23 of 375
angin_topan_0707 24 of 375
angin_topan_0731 25 of 375
angin_topan_0792 26 of 375
angin_topan_0812 27 of 375
angin_topan_0830 28 of 375
angin_topan_0867 29 of 375
angin_topan_0876 30 of 375
angin_topan_0902 31 of 375
angin_topan_0912 32 of 375
angin_topan_0963 33 of 375
angin_topan_0985 34 of 375
angin_topan_1006 35 of 375
angin_topan_1

tsunami_0293 338 of 375
tsunami_0345 339 of 375
tsunami_0351 340 of 375
tsunami_0386 341 of 375
tsunami_0404 342 of 375
tsunami_0414 343 of 375
tsunami_0436 344 of 375
tsunami_0463 345 of 375
tsunami_0478 346 of 375
tsunami_0501 347 of 375
tsunami_0513 348 of 375
tsunami_0537 349 of 375
tsunami_0551 350 of 375
tsunami_0588 351 of 375
tsunami_0593 352 of 375
tsunami_0620 353 of 375
tsunami_0654 354 of 375
tsunami_0675 355 of 375
tsunami_0708 356 of 375
tsunami_0735 357 of 375
tsunami_0760 358 of 375
tsunami_0782 359 of 375
tsunami_0794 360 of 375
tsunami_0864 361 of 375
tsunami_0906 362 of 375
tsunami_0918 363 of 375
tsunami_0939 364 of 375
tsunami_0964 365 of 375
tsunami_1011 366 of 375
tsunami_1065 367 of 375
tsunami_1069 368 of 375
tsunami_1104 369 of 375
tsunami_1107 370 of 375
tsunami_1142 371 of 375
tsunami_1146 372 of 375
tsunami_0536 373 of 375
tsunami_1109 374 of 375
selesai sebuah threshold
selesai sebuah eksperimen
Eksperimen 3 threshold 5
angin_topan_0005 0 of 375
angin_topa

longsor_0587 299 of 375
longsor_0617 300 of 375
longsor_0646 301 of 375
longsor_0675 302 of 375
longsor_0697 303 of 375
longsor_0716 304 of 375
longsor_0731 305 of 375
longsor_0765 306 of 375
longsor_0798 307 of 375
longsor_0818 308 of 375
longsor_0840 309 of 375
longsor_0866 310 of 375
longsor_0877 311 of 375
longsor_0910 312 of 375
longsor_0927 313 of 375
longsor_0956 314 of 375
longsor_0966 315 of 375
longsor_0998 316 of 375
longsor_1022 317 of 375
longsor_1042 318 of 375
longsor_1083 319 of 375
longsor_1109 320 of 375
longsor_1124 321 of 375
longsor_1172 322 of 375
longsor_1181 323 of 375
longsor_1204 324 of 375
longsor_0192 325 of 375
longsor_1131 326 of 375
tsunami_0004 327 of 375
tsunami_0024 328 of 375
tsunami_0064 329 of 375
tsunami_0098 330 of 375
tsunami_0123 331 of 375
tsunami_0156 332 of 375
tsunami_0165 333 of 375
tsunami_0201 334 of 375
tsunami_0208 335 of 375
tsunami_0245 336 of 375
tsunami_0279 337 of 375
tsunami_0293 338 of 375
tsunami_0345 339 of 375
tsunami_0351 340

kekeringan_0747 261 of 375
kekeringan_0773 262 of 375
kekeringan_0797 263 of 375
kekeringan_0829 264 of 375
kekeringan_0848 265 of 375
kekeringan_0892 266 of 375
kekeringan_0941 267 of 375
kekeringan_0993 268 of 375
kekeringan_1068 269 of 375
kekeringan_1080 270 of 375
kekeringan_0130 271 of 375
kekeringan_1086 272 of 375
kekeringan_0378 273 of 375
longsor_0002 274 of 375
longsor_0034 275 of 375
longsor_0055 276 of 375
longsor_0123 277 of 375
longsor_0139 278 of 375
longsor_0164 279 of 375
longsor_0199 280 of 375
longsor_0221 281 of 375
longsor_0235 282 of 375
longsor_0248 283 of 375
longsor_0275 284 of 375
longsor_0303 285 of 375
longsor_0308 286 of 375
longsor_0335 287 of 375
longsor_0350 288 of 375
longsor_0389 289 of 375
longsor_0418 290 of 375
longsor_0435 291 of 375
longsor_0463 292 of 375
longsor_0480 293 of 375
longsor_0498 294 of 375
longsor_0510 295 of 375
longsor_0544 296 of 375
longsor_0561 297 of 375
longsor_0579 298 of 375
longsor_0587 299 of 375
longsor_0617 300 of 375
l

karhutla_1018 225 of 375
karhutla_1040 226 of 375
karhutla_1059 227 of 375
karhutla_1078 228 of 375
karhutla_1107 229 of 375
karhutla_1119 230 of 375
karhutla_1137 231 of 375
karhutla_0232 232 of 375
karhutla_0904 233 of 375
karhutla_0080 234 of 375
kekeringan_0020 235 of 375
kekeringan_0034 236 of 375
kekeringan_0063 237 of 375
kekeringan_0129 238 of 375
kekeringan_0137 239 of 375
kekeringan_0187 240 of 375
kekeringan_0224 241 of 375
kekeringan_0239 242 of 375
kekeringan_0279 243 of 375
kekeringan_0288 244 of 375
kekeringan_0340 245 of 375
kekeringan_0385 246 of 375
kekeringan_0389 247 of 375
kekeringan_0406 248 of 375
kekeringan_0439 249 of 375
kekeringan_0447 250 of 375
kekeringan_0481 251 of 375
kekeringan_0497 252 of 375
kekeringan_0528 253 of 375
kekeringan_0556 254 of 375
kekeringan_0587 255 of 375
kekeringan_0620 256 of 375
kekeringan_0654 257 of 375
kekeringan_0682 258 of 375
kekeringan_0698 259 of 375
kekeringan_0720 260 of 375
kekeringan_0747 261 of 375
kekeringan_0773 262 o

In [193]:
hasil_eksperimen = pd.DataFrame([], columns=['id', 'TP', 'FP', 'FN', 'Prec', 'Rec','F1'])
for i in range(len(df_all_test)):
    id_berita = df_all_test['id'].iloc[i]
    print(id_berita, f'{i} of {len(df_all_test)}')
    tempe_df = get_df_hasil_eval_each_doc(id_berita, eksperimen,threshold)
    hasil_eksperimen= pd.concat([hasil_eksperimen, tempe_df], ignore_index=True)
hasil_eksperimen.to_csv(f'./prediction/result_information_extraction/Hasil_Evaluasi_TEST_NER.csv',index = False)

angin_topan_0005 0 of 375
angin_topan_0040 1 of 375
angin_topan_0051 2 of 375
angin_topan_0066 3 of 375
angin_topan_0107 4 of 375
angin_topan_0143 5 of 375
angin_topan_0144 6 of 375
angin_topan_0181 7 of 375
angin_topan_0203 8 of 375
angin_topan_0206 9 of 375
angin_topan_0224 10 of 375
angin_topan_0244 11 of 375
angin_topan_0297 12 of 375
angin_topan_0301 13 of 375
angin_topan_0356 14 of 375
angin_topan_0445 15 of 375
angin_topan_0470 16 of 375
angin_topan_0491 17 of 375
angin_topan_0543 18 of 375
angin_topan_0562 19 of 375
angin_topan_0583 20 of 375
angin_topan_0625 21 of 375
angin_topan_0677 22 of 375
angin_topan_0696 23 of 375
angin_topan_0707 24 of 375
angin_topan_0731 25 of 375
angin_topan_0792 26 of 375
angin_topan_0812 27 of 375
angin_topan_0830 28 of 375
angin_topan_0867 29 of 375
angin_topan_0876 30 of 375
angin_topan_0902 31 of 375
angin_topan_0912 32 of 375
angin_topan_0963 33 of 375
angin_topan_0985 34 of 375
angin_topan_1006 35 of 375
angin_topan_1026 36 of 375
angin_topan

tsunami_0345 339 of 375
tsunami_0351 340 of 375
tsunami_0386 341 of 375
tsunami_0404 342 of 375
tsunami_0414 343 of 375
tsunami_0436 344 of 375
tsunami_0463 345 of 375
tsunami_0478 346 of 375
tsunami_0501 347 of 375
tsunami_0513 348 of 375
tsunami_0537 349 of 375
tsunami_0551 350 of 375
tsunami_0588 351 of 375
tsunami_0593 352 of 375
tsunami_0620 353 of 375
tsunami_0654 354 of 375
tsunami_0675 355 of 375
tsunami_0708 356 of 375
tsunami_0735 357 of 375
tsunami_0760 358 of 375
tsunami_0782 359 of 375
tsunami_0794 360 of 375
tsunami_0864 361 of 375
tsunami_0906 362 of 375
tsunami_0918 363 of 375
tsunami_0939 364 of 375
tsunami_0964 365 of 375
tsunami_1011 366 of 375
tsunami_1065 367 of 375
tsunami_1069 368 of 375
tsunami_1104 369 of 375
tsunami_1107 370 of 375
tsunami_1142 371 of 375
tsunami_1146 372 of 375
tsunami_0536 373 of 375
tsunami_1109 374 of 375


# END of Experiment Location Extraction

### Below code just for self evaluation why the result is still not good enough.
The code probably not runable:)

In [40]:
%cd /workspace/dataset

/workspace/dataset


In [42]:
df_input = pd.read_csv(f'./angin_topan/angin_topan_train.csv')
lst_part = ['train' for i in range(len(df_input))]
for bencana in ['angin_topan', 'banjir', 'erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']:
    for partition in ['train', 'dev', 'test']:
        if bencana =='angin_topan' and partition == 'train':
            continue
        temp_df = pd.read_csv(f'./{bencana}/{bencana}_{partition}.csv')
        df_input = pd.concat([df_input, temp_df], ignore_index=True)
        for i in range(len(temp_df)):
            lst_part.append(partition)
df_input['part'] = lst_part
df_input = df[['id','tc_ori','label', 'part']]
df_input

,id,tc_ori,label,part
0,angin_topan_0002,Pencarian Nelayan Simeulue Dihentikan. Setelah...,Y,train
1,angin_topan_0003,Pohon Tumbang Timpa Warga Korban Meninggal Dal...,Y,train
2,angin_topan_0004,Ulama Tetap Menjadi Referensi KPA/PA. Ulama te...,N,train
3,angin_topan_0006,"Prihatin, KAHMI Cari Dana Bantu Korban Puting ...",Y,train
4,angin_topan_0007,Dua Rumah Warga Nagan Raya Ketimpa Pohan. Akib...,Y,train
...,...,...,...,...
9388,tsunami_0323,"Percepat Penanganan Bencana, Uu Ruzhanul Ulum ...",N,test
9389,tsunami_0364,60% BTS yang Rusak Akibat Gempa dan Tsunami Pa...,N,test
9390,tsunami_0010,"Gempa 6,4 SR Guncang Jatim dan Bali. Gempa den...",N,test
9391,tsunami_1109,Dylan Sahara Masih Hilang Pasca Tsunami Banten...,Y,test


### Verify NER Grit Model performance in this research
objective: makesure that NER grit can retrieve all of location in text

In [112]:
df_all_test

id  \
1010  angin_topan_0005   
1013  angin_topan_0040   
1019  angin_topan_0051   
1022  angin_topan_0066   
1028  angin_topan_0107   
...                ...   
9373      tsunami_1107   
9376      tsunami_1142   
9379      tsunami_1146   
9386      tsunami_0536   
9391      tsunami_1109   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [113]:
df_gt

,id,Positif Bencana Alam,Tanggal Kejadian,Lokasi ADM1,Lokasi ADM2,Lokasi ADM3,Lokasi Lainnya,loc_aggregate
0,angin_topan_0001,O,2018-09-30,Aceh,Bener Meriah,Wih Pesam,Kampung Wih Pesam,"(WIH PESAM, BENER MERIAH, ACEH)"
1,angin_topan_0002,O,2016-11-04,Aceh,Simeulue,Simeulue Timur,Perairan laut pulau Simanaha,"(SIMEULUE TIMUR, SIMEULUE, ACEH)"
2,angin_topan_0003,O,2018-01-03,Aceh,Sabang,Sukakarya Sabang,Gampong Kuta Timu,"(SUKAKARYA SABANG, SABANG, ACEH)"
4,angin_topan_0005,O,2017-04-09,Aceh,Simeulue,Simeulue Timur,Desa Amaiteng Mulia,"(SIMEULUE TIMUR, SIMEULUE, ACEH)"
5,angin_topan_0006,O,2019-08-18,Aceh,Aceh Barat,Sungai Mas,Desa Lancong,"(SUNGAI MAS, ACEH BARAT, ACEH)"
...,...,...,...,...,...,...,...,...
14096,tsunami_1154,O,2018-09-28,Sulawesi Tengah,Donggala,UNKNOWN,NaN,"(UNKNOWN, DONGGALA, SULAWESI TENGAH)"
14100,tsunami_1158,O,NaN,Sulawesi Tengah,Palu,UNKNOWN,NaN,"(UNKNOWN, PALU, SULAWESI TENGAH)"
14101,tsunami_1158,O,NaN,Sulawesi Tengah,Donggala,UNKNOWN,NaN,"(UNKNOWN, DONGGALA, SULAWESI TENGAH)"
14103,tsunami_1160,O,2018-12-22,Banten,UNKNOWN,UNKNOWN,NaN,"(UNKNOWN, UNKNOWN, BANTEN)"


In [ ]:
list_gt_test = []
st_kosong = ""

In [139]:
hasil_ner_grit = pd.DataFrame([], columns=['id', 'TP', 'FN', 'FP', 'TN','Prec', 'Rec','F1','ACC'])
for identitas in list(df_all_test['id']):
    partial_loc_agg_with_id = list(df_gt[df_gt['id']==identitas]['loc_aggregate'])
    all_loc_in_id = set()
    for lokasi_gt_complete in partial_loc_agg_with_id:
        for tk_adm in lokasi_gt_complete:
            tk_adm_split = tk_adm.split()
            for tkadm in tk_adm_split:
                all_loc_in_id.add(tkadm)
    all_loc_in_id=all_loc_in_id-{'UNKNOWN', 'NULL'}
    print(all_loc_in_id)
    teks_id = list(df_all_test[df_all_test['id']==identitas]['tc_ori'])[0]
    
    list_of_sentencess = split_sentences(teks_id)
    #print(list_of_sentence)
    df_get_entitasss = get_hasil_ner_tiap_sentence(list_of_sentencess[0])
    

    for i in range(1, len(list_of_sentencess)):
        df_get_entitasss = pd.concat([df_get_entitasss, get_hasil_ner_tiap_sentence(list_of_sentencess[i])], ignore_index=True)
    
    
    
    df_sementara = df_get_entitasss
    TPs, FNs, FPs, TNs = 0,0,0,0
    for m in range(len(df_sementara)):
        name_lab = df_sementara['label'].iloc[m]
        katas = df_sementara['words'].iloc[m]
        if katas.upper() in all_loc_in_id:
            #print(name_lab)
            if 'PLACE' in name_lab.upper() :
                TPs+=1
            else:
                FNs+=1
        else:
            if 'PLACE' in name_lab.upper() :
                FPs+=1
            else:
                TNs+=1
    Precs, Recs, F1s, Accs= 0,0,0,0
    if TPs+FPs!=0:
        Precs = TPs/(TPs+FPs)
    if TPs+FNs!=0:
        Recs = TPs/(TPs+FNs)
    if Precs*Recs!=0:
        F1s = 2/(1/Precs+1/Recs)
    if TPs+FPs+FNs+TNs!=0:
        Accs = (TNs+TPs)/(TPs+FPs+FNs+TNs)
    print(TPs, FNs, FPs, TNs)
    tempe_hasil_ner_grit = pd.DataFrame([[identitas, TPs, FNs, FPs, TNs, Precs, Recs, F1s, Accs]], columns=['id', 'TP', 'FN', 'FP', 'TN','Prec', 'Rec','F1','ACC'])
    hasil_ner_grit= pd.concat([hasil_ner_grit, tempe_hasil_ner_grit], ignore_index=True)
    
hasil_ner_grit

{'TIMUR', 'SIMEULUE', 'ACEH'}
8 0 4 198
{'RAYA', 'BLANGPIDIE', 'NAGRAN', 'BARAT', 'BEUTONG', 'DAYA', 'ACEH', 'SUSOH'}
19 0 43 651
{'KERAMBITAN', 'BALI', 'TABANAN'}
5 0 28 297
{'BALI', 'RENDANG', 'KARANGASEM'}
7 0 36 322
{'SELATAN', 'BARITO', 'KALIMANTAN', 'TABUKAN', 'KUALA'}
6 0 39 453
{'CARENANG', 'SERANG', 'BANTEN'}
9 0 13 288
{'BANTEN', 'TANGERANG', 'MAUK'}
7 2 12 387
set()
0 0 0 95
{'SUNGAI', 'KEPULAUAN', 'BEDUK', 'BATAM', 'RIAU'}
1 0 4 165
{'LUAR', 'NEGERI'}
0 0 10 376
{'TABA', 'PENANJUNG', 'TENGAH', 'BENGKULU'}
10 0 22 315
{'SUKARAJA', 'SELUMA', 'BENGKULU'}
6 0 9 258
{'MOOTILANGO', 'GORONTALO'}
6 0 10 210
{'CIAMIS', 'BARAT', 'JAWA', 'PURWADADI'}
0 0 6 64
{'JAWA', 'KABUPATEN', 'BARAT', 'RANCAEKEK', 'BANDUNG'}
18 0 17 332
{'BOYOLALI', 'JAWA', 'SELO', 'TENGAH'}
5 0 11 271
{'SURABAYA', 'JAWA', 'GUBENG', 'TIMUR'}
9 0 9 122
{'TEMPEL', 'PRAMBANAN', 'DEPOK', 'SEYEGAN', 'CANGKRINGAN', 'YOGYAKARTA', 'BERBAH', 'SLEMAN', 'BANTUL', 'KALASAN', 'MINGGIR'}
32 0 44 516
{'SELATAN', 'TAPIN', 'LARAS

4 0 10 259
{'BEBANDEM', 'BALI', 'RENDANG', 'KUBU', 'SELAT', 'KARANGASEM'}
10 0 28 258
{'TRIRENGGO', 'BANTUL', 'YOGYAKARTA'}
3 1 7 153
{'TENGGARA', 'BALI', 'BARAT', 'NUSA'}
6 1 41 357
{'SUMBAWA', 'LOMBOK', 'BARAT', 'BESAR', 'UTARA', 'TIMUR', 'TENGGARA', 'NUSA'}
22 0 2 488
{'PALU', 'SULAWESI', 'DONGGALA', 'TENGAH'}
21 0 15 578
{'SIGI', 'SULAWESI', 'DONGGALA', 'TENGAH', 'PALU'}
16 4 14 625
{'PALU', 'SULAWESI', 'DONGGALA', 'TENGAH'}
6 0 8 166
{'PALU', 'SULAWESI', 'DONGGALA', 'TENGAH'}
20 0 9 422
{'JAWA', 'DESA', 'MARUYUNGSARI', 'KALIPUCANG', 'KEBUMEN', 'TENGAH', 'PADAHERANG'}
16 1 18 444
{'PALU', 'SULAWESI', 'DONGGALA', 'TENGAH'}
10 0 1 655
{'LOMBOK', 'BARAT', 'UTARA', 'TENGGARA', 'NUSA'}
11 1 8 341
{'PALU', 'SULAWESI', 'DONGGALA', 'TENGAH'}
4 0 19 201
{'BARAT', 'JAWA', 'SUKABUMI', 'KABUPATEN'}
6 0 3 196
{'JAWA', 'KABUPATEN', 'CIDAHU', 'BARAT', 'PARAKANSALAH', 'KALAPANUNGGAL', 'SUKABUMI', 'KABANDUNGAN'}
17 0 1 155
{'JAWA', 'KABUPATEN', 'BARAT', 'BANDUNG', 'PANGALENGAN'}
13 0 23 351
{'PALU'

24 0 12 329
{'BANJARNEGARA', 'JAWA', 'MANDIRAJA', 'TENGAH'}
5 1 3 173
{'HAJI', 'SAPE', 'LOMBOK', 'SEKOTONG', 'SAMBALIA', 'TANO', 'BATU', 'LAPE', 'UTAN', 'SETELUK', 'UTARA', 'TIMUR', 'KOTA', 'POTO', 'LOPOK', 'EMPANG', 'KAYANGAN', 'KEMPO', 'KILO', 'LENANGGUAR', 'LEMBAR', 'LANTEH', 'NUSA', 'SUMBAWA', 'BARAT', 'LUNYUK', 'SAKRA', 'DAYA', 'DOMPU', 'MALUK', 'TENGAH', 'TENGGARA', 'KERUAK', 'PRAYA', 'JANAPRIA', 'PLAMPANG', 'BAYAN', "HU'U", 'AMBALAWI', 'KABUPATEN', 'TALIWANG', 'JEREWEH', 'BIMA', 'ALAS', 'PUJUT'}
52 9 41 387
{'MERAUKE', 'PAPUA', 'SEMANGGA'}
5 0 14 268
{'PAPUA', 'SORONG', 'AIMAS', 'BARAT'}
4 0 8 332
{'JAWA', 'BULU', 'NGUTER', 'TENGAH', 'WERU', 'BENDOSARI', 'SUKOHARJO'}
9 0 8 212
{'LANDAWE', 'SULAWESI', 'KONAWE', 'UTARA', 'TENGGARA'}
3 0 7 268
{'BARAT', 'LUBUK', 'SUMATERA', 'KILANGAN', 'PADANG', 'BEGALUNG'}
15 5 49 315
{'SAMOSIR', 'SUMATERA', 'PALIPI', 'SIMANINDO', 'UTARA'}
10 0 9 307
{'JAWA', 'BLITAR', 'BAKUNG', 'WONOTIRTO', 'TIMUR', 'PANGGUNGREJO'}
13 0 14 417
{'PATIA', 'SUKARESM

,id,TP,FN,FP,TN,Prec,Rec,F1,ACC
0,angin_topan_0005,8,0,4,198,0.666667,1.0,0.800000,0.980952
1,angin_topan_0040,19,0,43,651,0.306452,1.0,0.469136,0.939691
2,angin_topan_0051,5,0,28,297,0.151515,1.0,0.263158,0.915152
3,angin_topan_0066,7,0,36,322,0.162791,1.0,0.280000,0.901370
4,angin_topan_0107,6,0,39,453,0.133333,1.0,0.235294,0.921687
...,...,...,...,...,...,...,...,...,...
370,tsunami_1107,2,0,11,352,0.153846,1.0,0.266667,0.969863
371,tsunami_1142,9,0,3,129,0.750000,1.0,0.857143,0.978723
372,tsunami_1146,13,0,63,1092,0.171053,1.0,0.292135,0.946062
373,tsunami_0536,6,0,26,297,0.187500,1.0,0.315789,0.920973


In [143]:
def get_score_from_df(df):
    df['acc'] = (df['TP']+df['TN'])/(df['TN']+df['TP']+df['FN']+df['FP'])
    df = df[df['acc'].notnull()]
    #print(df[df['acc'].isnull()])
    print(f"Macro ACC = {round(sum(df['acc'])/len(df),3)} F1-Score = {round(sum(df['F1'])/len(df),3)} Prec = {round(sum(df['Prec'])/len(df),3)} Rec = {round(sum(df['Rec'])/len(df),3)} ")
    mic_prec = sum(df['TP'])/(sum(df['TP'])+sum(df['FP']))
    mic_rec = sum(df['TP'])/(sum(df['TP'])+sum(df['FN']))
    mic_f1 = 2*mic_prec*mic_rec/(mic_prec+mic_rec)
    mic_acc = (sum(df['TN'])+sum(df['TP']))/(sum(df['TN'])+sum(df['TP'])+sum(df['FP'])+sum(df['FN']))
    print(f"Micro ACC = { round(mic_acc,3)} F1-Score = {round(mic_f1,3)} Prec = {round(mic_prec,3)} Rec = {round(mic_rec,3)}")

#### result of ner grit
the result shows that there are some **False Positive**

In [144]:
hasil_ner_grit

,id,TP,FN,FP,TN,Prec,Rec,F1,ACC
0,angin_topan_0005,8,0,4,198,0.666667,1.0,0.800000,0.980952
1,angin_topan_0040,19,0,43,651,0.306452,1.0,0.469136,0.939691
2,angin_topan_0051,5,0,28,297,0.151515,1.0,0.263158,0.915152
3,angin_topan_0066,7,0,36,322,0.162791,1.0,0.280000,0.901370
4,angin_topan_0107,6,0,39,453,0.133333,1.0,0.235294,0.921687
...,...,...,...,...,...,...,...,...,...
370,tsunami_1107,2,0,11,352,0.153846,1.0,0.266667,0.969863
371,tsunami_1142,9,0,3,129,0.750000,1.0,0.857143,0.978723
372,tsunami_1146,13,0,63,1092,0.171053,1.0,0.292135,0.946062
373,tsunami_0536,6,0,26,297,0.187500,1.0,0.315789,0.920973


In [145]:
get_score_from_df(hasil_ner_grit)

Macro ACC = 0.943 F1-Score = 0.481 Prec = 0.348 Rec = 0.937 
Micro ACC = 0.947 F1-Score = 0.492 Prec = 0.33 Rec = 0.966


### verify performance of gazetteer
objective: investigate is it any error that causes by gazetteer

In [90]:
len(set(list(df_gt['loc_aggregate'])))

2018

In [93]:
all_locc = list(set(list(df_gt['loc_aggregate'])))
all_locc[:100]

[('GAJAH', 'DEMAK', 'JAWA TENGAH'),
 ('DUMAI SELATAN', 'DUMAI', 'RIAU'),
 ('UNKNOWN', 'LAMPUNG TIMUR', 'LAMPUNG'),
 ('JAMBLANG', 'CIREBON', 'JAWA BARAT'),
 ('PANGKALAN', 'KARAWANG', 'JAWA BARAT'),
 ('SAUSU', 'PARIGI MOUTONG', 'SULAWESI TENGAH'),
 ('UNKNOWN', 'UNKNOWN', 'KEPULAUAN BANGKA BELITUNG'),
 ('CIJERUK', 'KABUPATEN BOGOR', 'JAWA BARAT'),
 ('SUMPUR KUDUS', 'SIJUNJUNG', 'SUMATERA BARAT'),
 ('MAUPONGGO', 'NAGEKEO', 'NUSA TENGGARA TIMUR'),
 ('UNKNOWN', 'TANGERANG SELATAN', 'BANTEN'),
 ('MANTANGAI', 'KAPUAS', 'KALIMANTAN TENGAH'),
 ('MANDIRAJA', 'BANJARNEGARA', 'JAWA TENGAH'),
 ('NGUTER', 'SUKOHARJO', 'JAWA TENGAH'),
 ('UNKNOWN', 'JAKARTA UTARA', 'JAKARTA'),
 ('MELAYA', 'KARANGASEM', 'BALI'),
 ('UNKNOWN', 'SINJAI', 'SULAWESI SELATAN'),
 ('UNKNOWN', 'NGAWI', 'JAWA TIMUR'),
 ('MAMOSALATO', 'MOROWALI UTARA', 'SULAWESI TENGAH'),
 ('BAGAN SINEMBAH', 'ROKAN HILIR', 'RIAU'),
 ('SEUNAGAN', 'NAGAN RAYA', 'ACEH'),
 ('SOLEAR', 'KABUPATEN TANGERANG', 'BANTEN'),
 ('UNKNOWN', 'PURWOREJO', 'JAWA TE

### There is some Kecamatan in Ground Truth but not provided by gazetteer
Kecamatan pada Ground Truth yang tidak ada pada gazeteer

In [100]:
banyak_kec_none = 0
for kecamatans in all_locc:
    kecc = kecamatans[0]
    if kecc == "UNKNOWN" or kecc== 'NULL':
        continue
    try:
        kec_to_id[kecc]
    except:
        banyak_kec_none+=1
        print(kecc)
print(banyak_kec_none)

TELUK BAYAR
KECAMATAN DUKUN
SINTANG SUNGAI TEBELIAN
LOWOKMARU
LEBAK GEDONG
TOPOLS
TIRTAYAS
SANGTOMBOLANG
TANAH SEREAL
V KOTO KAMPUANG DALAM
JANGKABUYA
MANDAU TELAWANG
LEMBAH AGRO
TANJUNG TEBAT
DESA WAISAMU
SOSBAANG
LINGGO SARI
MANGGARAI SELATAN
JAUI
SAKUPANG
KLIKIM TIMUR
PALMERAH
SAGAH
BAKTIYA 
SIDANGRESMI
TAPAK TUAN
WANAYAS
PENDAMARAN TIMUR
RILAU ALE
POLI-POLIA
TELUK BETUNG UTARA
TANAH JAMBO AYE 
BAYUNGLENCIR
CURUGBITUNG
DESA SANGTOMBOLANG
KRAMAT JATI
LUBUK BAJAR
DESA WAAI
TEGALDIMO
CIPEDANG
LUBUKLINGGAU BARAT II
WONOSOLAM
SENANDWETAN
PIRAK TIMU 
PONDOK SALAM
NULL 
PULAU PINANG
MEUNASAH JURONG
SENAYAN
SUKAKARYA SABANG
SILIAN
PADANG ULAK TENDING
KECAMATAN PENGKADAN
CIMANGGI
JL. PERINTIS KEMERDEKAAN
PULAU LAUT UTARA
TIGA NDERKET
HUJAN MAS
TEGALALANG
TRIENG GADENG
KEBANDUNGAN
MADAPANGA
TAWAEILI
GILI GENTING
ATAMBUA
WATTANG SIDENRENG
DESA IRAT
DEMAK KOTA
SALIMPAUNG
KUROK
PAMULUTAN
KOMOPLEKS DUTA HARAPAN INDAH
BUSUNGBIU
RAWA BUAYA
GUNDIH
JAMPANG TENGAH
PENTERAJA
MANGGELEWA
TARKAN TENGAH
PA

### There is some Kota/City in Ground Truth but not provided by gazetteer
kota pada ground truth yang tidak ada pada gazeteer

In [104]:
banyak_kota_none = 0
lst_kota = set()
for kecamatans in all_locc:
    kecc = kecamatans[1]
    lst_kota.add(kecc)
all_loc = list(lst_kota)
for kecc in all_loc:
    if kecc == "UNKNOWN" or kecc== 'NULL':
        continue
    try:
        city_to_id[kecc]
    except:
        banyak_kota_none+=1
        print(kecc)
print(banyak_kota_none)

TANAH BAMBU
BANKA SELATAN
KEPULAUAN SIAU TAGULANDANG BIARO
KABUPATEN KEPULAUAN SANGIHE
PAGARALAM
KABUPATEN KARANGANYAR
KABUPATEN TELUK BINTUNI
KOTA DUMAI
KOTA TANGERANG
KABUPATEN MAMUJU
MUAROJAMBI
KABUPATEN TASIKMALAYA
KABUPATEN SANGGAU
SUMENEH
TIDORE
KABUPATEN MAGELANG
KABUPATEN JEMBER
WINONGAN
KABUPATEN MAGETAN
KOTA KUBU RAYA
SUMBAWA BESAR
BAUBAU
KOTA SEMARANG
KEPULAUAN MERAU
MUARAENIM
ILE APE
KABUPATEN PURWAKARTA
NAGRAN RAYA
MUSI RAWA UTARA
PENJARINGAN
MARTAPURA
GUNUNG KIDUL
KABUPATEN SIGI
KABUPATEN BANJERNEGARA
KABUPATEN MOJOKERTO
KABUPATEN LANDAK
KARARNGASEM
PASAR REBO
KABUPATEN MATENG
KOTA BOGOR
TOBASA
TANJUNGPINANG
DHARMASRAYA   TIMPEH
EMPATLAWANG
ACEH
KABUPATEN BIMA
SAMBAR
KOTA MATARAM
TIMOR TENGAH SELATAN
LEBANG
SELAYAR
KOTA PARIAMAN
BATUSANGKAR
BALI
KOTAWARINGIN TIMU
MAKASAR
KOTA MOJOKERTO
KOTA BENGKULU
KABUPATEN MALANG
TOJO UNA-UNA
DOMPA
ILE APE TIMUR
SANGIHE
DELISERDANG
KABUPATEN SELAYAR
KOTA LUBUKLINGGAU
KABUPATEN KAPUAS HULU
KOTA BLITAR
LIMAPULUH KOTA
KABUPATEN GORONTALO


### there are some inconsistent in gazetteer
- there are some kecamatan that has name *kecamatan* or *kota* as prefix, however the others are not


In [110]:
un_consistent = 0
for lokasi_daerah in kec_to_id.keys():
    if ('KECAMATAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KOTA' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KABUPATEN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KELURAHAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('JALAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
print(un_consistent)

KOTA BAHAGIA
KOTA JANTHO
KOTA COT GLIE
KOTA SIGLI
KOTA BAHARU
KOTA JUANG
KOTA KUALASINPANG
LANGSA KOTA
KOTA KISARAN BARAT
KOTA KISARAN TIMUR
KOTANOPAN
KOTARIH
KOTAPINANG
MEDAN KOTA
SIBOLGA KOTA
BINJAI KOTA
TEBING TINGGI KOTA
BANGKINANG KOTA
PEKANBARU KOTA
DUMAI KOTA
JAMBI LUAR KOTA
SEBERANG KOTA
KOTA BARU
KOTA AGUNG
KOTA MANNA
KOTA PADANG
KOTA ARGA MAKMUR
KOTA MUKOMUKO
KOTA GAJAH
KOTABUMI
KOTABUMI UTARA
KOTABUMI SELATAN
KOTA AGUNG BARAT
KOTA AGUNG TIMUR
BATAM KOTA
TANJUNG PINANG KOTA
GARUT KOTA
JALANCAGAK
KOTA KUDUS
KOTAGEDE
KOTAANYAR
KOTA SUMENEP
KOTA
ASAKOTA
KOTA SOE
KOTA KEFAMENANU
KOTA ATAMBUA
KOTA WAINGAPU
KOTA WAIKABUBAK
KOTA TAMBOLAKA
KOTA KOMBA
KOTA RAJA
KOTA LAMA
PONTIANAK KOTA
KOTAWARINGIN LAMA
KOTA BESI
KOTA BANGUN
BALIKPAPAN KOTA
SAMARINDA KOTA
MALINAU KOTA
KOTABUNAN
KOTAMOBAGU UTARA
KOTAMOBAGU TIMUR
KOTAMOBAGU SELATAN
KOTAMOBAGU BARAT
POSO KOTA
POSO KOTA SELATAN
POSO KOTA UTARA
AMPANA KOTA
KOTA BARAT
KOTA SELATAN
KOTA UTARA
KOTA TIMUR
KOTA TENGAH
KOTA MASOHI
KOTA MABA
KOTA

In [108]:
un_consistent = 0
for lokasi_daerah in city_to_id.keys():
    if ('KECAMATAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KOTA' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KABUPATEN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('KELURAHAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
    if ('JALAN' in lokasi_daerah):
        print(lokasi_daerah)
        un_consistent +=1
print(un_consistent)

LIMA PULUH KOTA
KOTAWARINGIN BARAT
KOTAWARINGIN TIMUR
KOTABARU
KOTAMOBAGU
5
